### Imports

In [2]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

from ImportUtils import get_dfInfoEscolas_from_parquet, get_dfInfoEscolas_for_analysis_from_parquet
from ImportUtils import vprint, vprint_time

from ChartUtils import chart
#from ChartUtils import linechart, barchart, barchart_nseries
#from main import dicParams, dicFiles

from main import dicAnalysis


K_MATPLOTLIB_STYLES_FILE = "NOVAENES.mplstyle"


### Load data

In [3]:
dfInfoEscolas_for_analysis = get_dfInfoEscolas_for_analysis_from_parquet()
print("Number of records;", dfInfoEscolas_for_analysis.shape[0], "; number of attributes:", dfInfoEscolas_for_analysis.shape[1])

print("Columns:")
print(dfInfoEscolas_for_analysis.columns.to_list())


Loaded Parquet file  C:/ENES/Parquet/dfInfoEscolas_for_analysis.parquet.gzip shape:  (1348692, 24)
Number of records; 1348692 ; number of attributes: 24
Columns:
['ano', 'Escola', 'Exame', 'Idade', 'Curso', 'CIF', 'Class_Exam', 'Class_Exam_Rounded', 'Distrito', 'Concelho', 'DescrNuts3', 'DescrNuts2', 'ExamRounded_median_CIF', 'ExamRounded_mean_CIF', 'ExamRounded_count_CIF', 'CIF_bonus_median_ExamRounded', 'CIF_bonus_mean_ExamRounded', 'CIF_bonus_adj_median_ExamRounded', 'CIF_bonus_adj_mean_ExamRounded', 'ExamOverallCount', 'ExamOverallRank', 'ExamYearlyRank', 'isFemale', 'isPublic']


In [5]:

dfInfoEscolas_for_analysis["ano"].groupby(dfInfoEscolas_for_analysis ["ano"])


In [6]:
for col in dfInfoEscolas_for_analysis.columns:
	xUniqueValues = dfInfoEscolas_for_analysis[col].unique()
	print(col, xUniqueValues.shape[0], "unique values.")
	if xUniqueValues.shape[0] <=5 :
		print("   ", col, xUniqueValues)

#x = pd.DataFrame(dfInfoEscolas["ano"].groupby(dfInfoEscolas["ano"]).count())
#print(x[0:5])



ano 12 unique values.
Escola 680 unique values.
Exame 32 unique values.
Idade 34 unique values.
Curso 9 unique values.
CIF 11 unique values.
Class_Exam 96 unique values.
Class_Exam_Rounded 11 unique values.
Distrito 21 unique values.
Concelho 28 unique values.
DescrNuts3 26 unique values.
DescrNuts2 8 unique values.
ExamRounded_median_CIF 20 unique values.
ExamRounded_mean_CIF 2260 unique values.
ExamRounded_count_CIF 1026 unique values.
CIF_bonus_median_ExamRounded 34 unique values.
CIF_bonus_mean_ExamRounded 19095 unique values.
CIF_bonus_adj_median_ExamRounded 227 unique values.
CIF_bonus_adj_mean_ExamRounded 114734 unique values.
ExamOverallCount 32 unique values.
ExamOverallRank 32 unique values.
ExamYearlyRank 27 unique values.
isFemale 2 unique values.
    isFemale [ True False]
isPublic 2 unique values.
    isPublic [ True False]


In [27]:
dfInfoEscolas_for_analysis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1348692 entries, 0 to 1348691
Data columns (total 27 columns):
 #   Column                            Non-Null Count    Dtype  
---  ------                            --------------    -----  
 0   ano                               1348692 non-null  int64  
 1   Escola                            1348692 non-null  object 
 2   Exame                             1348692 non-null  object 
 3   Idade                             1348692 non-null  float64
 4   Curso                             1348692 non-null  object 
 5   SitFreq                           1348692 non-null  object 
 6   CIF                               1348692 non-null  float64
 7   Class_Exam                        1348692 non-null  float64
 8   Class_Exam_Rounded                1348692 non-null  int32  
 9   Distrito                          1348692 non-null  object 
 10  Concelho                          1348692 non-null  object 
 11  DescrNuts3                        134

In [54]:
for col in dfInfoEscolas.columns:
	x = pd.DataFrame(dfInfoEscolas[col].groupby(dfInfoEscolas[col]).count())
	print("col", col, x.shape, type(x))
	print( "   ", x[0:3])



col ano (12, 1) <class 'pandas.core.frame.DataFrame'>
             ano
ano         
2008  113616
2009  129477
2010  120623
col Escola (680, 1) <class 'pandas.core.frame.DataFrame'>
            Escola
Escola        
0002      2250
0031       417
0039       469
col Exame (32, 1) <class 'pandas.core.frame.DataFrame'>
           Exame
Exame       
138       23
239      101
501     6361
col Sexo (2, 1) <class 'pandas.core.frame.DataFrame'>
            Sexo
Sexo        
F     910867
M     640751
col Idade (35, 1) <class 'pandas.core.frame.DataFrame'>
    Empty DataFrame
Columns: [Idade]
Index: []
col Curso (16, 1) <class 'pandas.core.frame.DataFrame'>
            Curso
Curso        
060    120438
061     17114
062     44439
col SitFreq (2, 1) <class 'pandas.core.frame.DataFrame'>
             SitFreq
SitFreq         
AE        584749
ND        966869
col CIF (11, 1) <class 'pandas.core.frame.DataFrame'>
    Empty DataFrame
Columns: [CIF]
Index: []
col Class_Exam (106, 1) <class 'pandas.core.